#**Hà Phương nhưng đọc gì cũng được :V**
---

Thanks to Viblo.asia  
Main Ref: [Source from Viblo.asia](https://viblo.asia/p/ong-toan-vi-loc-ung-dung-deep-learning-tu-dong-sinh-ra-series-audio-truyen-ma-sieu-to-khong-lo-bJzKmwqkl9N?fbclid=IwAR0GMsb8-sgoRKgmSyre_Y8k-ycZ3vCQ4A-EpDOVnuWs6-tFMy0ItQhLdKo)  
Người hướng dẫn, thầy: Trần Cao Trưởng

---



## 0. Đặt vấn đề  
- Trong một ứng dụng trợ lý ảo, việc trợ lý ảo có thể nói chuyện và đọc văn bản là không thể thiếu. Tuy nhiên trong phạm vi nhóm này sẽ chỉ quan tâm tới việc đọc văn bản. Vậy bài toán chính của chúng ta sẽ là **TextToSpeech**
- Khảo sát: cho đến hiện nay có 3 phương pháp chính ([reference](https://www.ijraset.com/fileserve.php?FID=14075))
  - Articulatory Speech Synthesis
  - Formant Speech Synthesis
  - Concatenative Speech Synthesis (ghép nối)
- Công nghệ được sử dụng trong bài toán này: 
  - [Tacotron 2 ](https://github.com/NVIDIA/tacotron2)
- Các kiến thức liên quan:
  - [CNN](https://medium.com/@hon_14885/convolutional-neural-network-c%C3%A1i-nh%C3%ACn-t%E1%BB%95ng-quan-c6aaba265a39)
  - [Seq2Seq](https://nerophung.github.io/2019/05/18/nlp-p3)
- Các phần mềm liên quan:
  - autosub (có thể sử dụng Google API để làm bước speech2text trong việc chuẩn bị dữ liệu)
  - ffmpeg
- Khảo sát cá nhân:
  - Một số công nghệ được sử dụng gần đây trong bài toán Text2Speech có thể kể đến [Wavenet](https://deepmind.com/blog/article/wavenet-generative-model-raw-audio) và [Tacotron](https://google.github.io/tacotron/publications/tacotron2/) và đã được đưa vào ứng dụng và sử dụng cho những bài toán thực tế như Google Assistant ([ref](https://deepmind.com/blog/article/wavenet-launches-google-assistant)) 
  - So sánh: https://becominghuman.ai/into-a-better-speech-synthesis-technology-29411b64f2a2
- Mục tiêu chính của chúng ta sẽ là sử dụng Tacotron cho bài toán text2speech  
- Các bước thực hiện: tạo data--->training---->test

---



## I. Chuẩn bị dữ liệu
- Phần quan trọng nhất trong 1 bài toán ML hay DL đó chính là dữ liệu
- Cụ thể với bài toán text2speech là nguồn dữ liệu giọng đọc  
  



### 1.   Thu thập dữ liệu
  - Phương pháp thu thập dữ liệu chủ yếu là thu từ những người nổi tiếng, hoặc những người có giọng đọc hay (do có nguồn dữ liệu đọc đủ lớn và giọng đọc thích hợp với đại đa số mọi người), hoặc ta có thể tự tạo nguồn dữ liệu bằng cách tự thu âm hoặc thuê giọng đọc nhằm đạt lượng dữ liệu đủ lớn để training
  - Do hạn chế về công cụ cũng như ngân sách, chúng ta sẽ sử dụng giọng đọc có sẵn từ Internet (youtube, các nguồn audio...), cụ thể ở đây sẽ là giọng đọc của [**Hà Phương**](https://www.youtube.com/results?search_query=h%C3%A0+ph%C6%B0%C6%A1ng+%C4%91%E1%BB%8Dc+truy%E1%BB%87n)
  - Nguồn có chất lượng âm thanh khá tốt và nhiều: [Truyện ma by Hà Phương :V](https://www.youtube.com/watch?v=foGgoa9xIaY&list=PLy5Rvg2t_Ox0KHfDlmQGM0cEzIR4Z4OHh)
  - Công cụ sử dụng: 
      - t̶̶o̶̶o̶̶l̶ ̶c̶̶r̶̶a̶̶w̶̶l̶ ̶t̶̶ự̶ ̶v̶̶i̶̶ế̶̶t̶
      - [KTSpeechCrawler](https://github.com/EgorLakomkin/KTSpeechCrawler)


In [ ]:
#lib for crawl
!pip install pytube3
!mkdir data

In [ ]:
from pytube import YouTube
from pytube import Playlist
import re
import pytube
import os

def download_playlist(url, path_dir):
    playlist = Playlist(url)
    playlist._video_regex = re.compile(r"\"url\":\"(/watch\?v=[\w-]*)")
    #prints each video url, which is the same as iterating through playlist.video_urls
    for url in playlist:
        print(url)
        YouTube(url).streams.filter(only_audio=True).first().download(path_dir)
        print('download complete!')
        print("----------------------")

#playlist truyen doc by Ha Phuong
url = 'https://www.youtube.com/playlist?list=PLfKmBpK2q_5NMtTiI_5hbH2dyCLhMfxcZ'
path = 'data'
download_playlist(url, path)

### 2.   Sinh text cho dữ liệu
  - Download dữ liệu từ youtube sử dụng: [Autosub](https://github.com/agermanidis/autosub) (tích hợp gọi googleAPI speech-to-text v1 nhưng không hiệu quả hơn nhiều, chúng ta sẽ sử dụng google-speech-to-text-v2 trong thư viện này)



In [ ]:
import os
import re

data_path = '/Users/trongthanht3/PycharmProjects/text2speech/data'
sapi = 'gsv2'      #read git of autosub to learn more gcsv1=google-speech-api
lang_code = 'vi-VN'

def no_accent_vietnamese(s):
    s = s.lower()
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[ÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬ]', 'A', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[ÉÈẺẼẸÊẾỀỂỄỆ]', 'E', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[ÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢ]', 'o', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[ÍÌỈĨỊ]', 'i', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ÚÙỦŨỤƯỨỪỬỮỰ]', 'u', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('[ÝỲỶỸỴ]', 'y', s)
    s = re.sub('đ', 'd', s)
    s = re.sub('Đ', 'D', s)
    s = re.sub('[ \b\t]', '_', s)
    return s

#rename to make sure there's no space or special char in file name
def file_rename(data_path):
    for file in os.listdir(data_path):
        new_name = no_accent_vietnamese(file)
        print(new_name)
        os.rename(data_path+'/'+file, data_path+'/'+new_name)

#we need to rename sub file after gen sub
#because autosub add lang_code to end of file name
#then we can use sub to generate csv
def file_rename_after(data_path):
    for file in os.listdir(data_path):
        if (file[-3:] == 'srt'):
            os.rename(data_path+'/'+file, data_path+'/'+file[:-9]+"srt")

def autosub_gen_srt(data_path, sapi, lang_code):
    print("start generate sub from audio")
    for file in os.listdir(data_path):
        #check if sub exist
        print("renaming: ", file)
        name = file[:-3]+'vi-vn.srt'
        if os.path.isfile(data_path+'/'+name):
            print("sub existed!")
            print("---------------------------")
            continue
        elif (file[-3:] == 'srt'):
            print("wrong file type!")
            print("---------------------------")
            continue
        else:
            command = 'autosub' + ' -i ' + data_path + '/' + file + ' -sapi ' + sapi\
                + ' -S ' + lang_code
            print(command)
            os.system(command)
            print("done!")
        print("---------------------------")



# file_rename(data_path)
# autosub_gen_srt(data_path, sapi, lang_code)
file_rename_after(data_path)
print("gen completed!")

- Tách audio thành các segment

In [ ]:
import pysrt
from pydub import AudioSegment

audio_name = 'truyenma1.mp3'
sub_name = 'truyenma1.vi-vn.srt'
audio_outdir = '/content/bobo/'
csv_output = 'output.csv'

song = AudioSegment.from_file(audio_name)
subs = pysrt.open(sub_name, encoding='utf-8')

# Define lambda function convert time to miliseconds 
time_to_ms = lambda x: (x.hours*3600 + x.minutes * 60 + x.seconds) * 1000 + x.milliseconds

# Extract data 
with open(csv_output, 'w') as fd:
    for sub in subs:
        # Get start time, end time in miliseconds
        start_ms = time_to_ms(sub.start)
        end_ms = time_to_ms(sub.end)   
        # Audio extracted file name
        audio_extract_name = '{}/{}_{}_{}.wav'.format(audio_outdir, audio_name, start_ms, end_ms)
        text = str(sub.text)
        # Extract file
        extract = song[start_ms:end_ms]
        # Saving 
        extract.export(audio_extract_name, format="wav")
        # Write to csv file
        fd.write('{}|{}\n'.format(audio_extract_name, text))

### 3. Chuẩn hoá text  
- Về chuẩn hoá text có một số bước như convert từ chữ hoa về chữ thường, convert từ số sang chữ, convert thời gian, ngày tháng sang chữ






In [ ]:
import re 
from num2words import num2words
from unicodedata import normalize

def vi_num2words(num):
   return num2words(num, lang='vi')

def convert_time_to_text(time_string):
   # Support only hh:mm format
   try:
       h, m = time_string.split(":")
       time_string = vi_num2words(int(h)) + " giờ " + vi_num2words(int(m)) + " phút" 
       return time_string
   except:
       return None

def replace_time(text):
   # Define regex to time hh:mm
   result = re.findall(r'\d{1,2}:\d{1,2}|', text)
   match_list = list(filter(lambda x : len(x), result))

   for match in match_list:
       if convert_time_to_text(match):
           text = text.replace(match, convert_time_to_text(match))
   return text

def replace_number(text):
   return re.sub('(?P<id>\d+)', lambda m: vi_num2words(int(m.group('id'))), text)

def normalize_text(text):
   text = normalize("NFC", text)
   text = text.lower()
   text = replace_time(text)
   text = replace_number(text)
   return text


In [ ]:
normalize_text(text)

NameError: ignored

## III. Demo bằng một bộ dữ liệu đã chuẩn hoá

In [ ]:
#reset to get p100 for best performance
!nvidia-smi

Mon Oct 26 11:24:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -Uqq gdown
!gdown --id 1ZTYOdfrHx_WORDmH6GoVrz3uGHypqday
!mkdir dataset
!tar -C dataset -xzf viet-tts.tar.gz

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Downloading...
From: https://drive.google.com/uc?id=1ZTYOdfrHx_WORDmH6GoVrz3uGHypqday
To: /content/viet-tts.tar.gz
4.28GB [01:16, 56.2MB/s]


In [ ]:
#edited for vietnamese
!git clone https://github.com/trongthanht3/tacotron2

Cloning into 'tacotron2'...
remote: Enumerating objects: 411, done.
remote: Total 411 (delta 0), reused 0 (delta 0), pack-reused 411
Receiving objects: 100% (411/411), 2.70 MiB | 2.50 MiB/s, done.
Resolving deltas: 100% (209/209), done.


In [ ]:
!pip install -r tacotron2/requirements.txt

     |████████████████████████████████| 15.0MB 206kB/s 
     |████████████████████████████████| 110.5MB 27kB/s 
     |████████████████████████████████| 17.0MB 177kB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 1.5MB 58.0MB/s 
     |████████████████████████████████| 50.0MB 107kB/s 
     |████████████████████████████████| 235kB 59.0MB/s 
     |████████████████████████████████| 512kB 40.6MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 3.8MB 44.8MB/s 
  Created wheel for librosa: filename=librosa-0.6.0-cp36-none-any.whl size=1553495 sha256=fd7c60bfda237e589f6ba90b09550d6169e49b98c67280e0753239ea657e71d2
  Stored in directory: /root/.cache/pip/wheels/0d/19/fa/71097e2207df1cc613749f15b2f0b1972c167b36d6afc09d15
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=5dd90798098707343098290ff4e1ee172b9320b45b4fceba83f8f7b9f6e7d13e
  Stored in directory: /root/.cache/

In [ ]:
!pip install num2words
!pip install unicodedata

     |████████████████████████████████| 102kB 2.2MB/s 
ERROR: Could not find a version that satisfies the requirement unicodedata (from versions: none)
ERROR: No matching distribution found for unicodedata


In [ ]:
#make traning data and test data
import pandas as pd

data = pd.read_csv('dataset/meta_data.tsv', sep='\t', skiprows=1, quoting=3, error_bad_lines=False)

# Suffle
data = data.sample(frac=1)
data.tail()

,dataset/wav/tts_lines_part_06_line_00000.wav,Hở!
32986,dataset/wav/tts_lines_part_02_line_00857.wav,Bạn thân của vợ mày đấy.
11640,dataset/wav/tts_lines_part_15_line_00283.wav,"- Thôi đi, xin các bố trẻ!"
33621,dataset/wav/tts_lines_part_02_line_01492.wav,"Đã vay một người bạn, đã chịu lụy bạn khi bạn ..."
702,dataset/wav/tts_lines_part_06_line_00703.wav,- Thế cũng có lý.
3348,dataset/wav/tts_lines_part_19_line_01459.wav,"Trong sáu trăm người ấy, người nào cũng có nhi..."


In [ ]:
import re

# data['text2'] = data['text'].map(lambda x: re.sub(r'["^<>{}\"/|;:.,~!?@#$%^=&*\\]\\\\()\\[¿§«»ω⊙¤°℃℉€¥£¢¡®©0-9_+]*$"]', '', x))
# data[[1]] = data[[1]].map(lambda x: re.sub(r"[^\w\s]*", '', x))
data.iloc[:,1] = data.iloc[:,1].map(lambda x: re.sub(r"[^\w\s]*", '', x))
data.tail()

,dataset/wav/tts_lines_part_06_line_00000.wav,Hở!
21864,dataset/wav/tts_lines_part_07_line_01065.wav,Sự im lặng này đã tố cáo một tội của Duyên khô...
36132,dataset/wav/tts_lines_part_04_line_00231.wav,Bẩm toàn dân tỉnh này đã bắt đầu nói xấu và v...
28606,dataset/wav/tts_lines_part_05_line_00250.wav,tôi không nghĩ gì cả vì có muốn nghĩ cũng khôn...
24856,dataset/wav/tts_lines_part_16_line_00279.wav,Cho nên thoạt đầu anh chỉ thấy vui vẻ cái khoa...
26030,dataset/wav/tts_lines_part_16_line_01453.wav,Cái khoái lạc ấy ở đời mà ai cũng bảo là nhất ...


In [ ]:
import numpy as np

df = data.iloc[:,1]

In [ ]:
df = df.astype('str')
df = df.astype('str')
mask = (df.str.len() <= 3) & (df.str.len() <= 3)
df2 = df.loc[mask]
print(df2)

7629      Dạ
35223    thì
9170       V
1345     Alo
5265      Hở
        ... 
28827     Dạ
32022      v
2615      Ờờ
34135     Dạ
3164      Ồồ
Name: Hở!, Length: 444, dtype: object


In [ ]:
#tao test.txt va training.txt

import os

train_ratio = 0.8
train_index = int(train_ratio * len(data))

with open('training.txt', 'w') as fd:
    for i, fname in enumerate(data.iloc[:,0][:train_index]):
        fd.write('{}|{}\n'.format(os.path.join(os.getcwd(), fname), data.iloc[:,1][i]))

with open('testing.txt', 'w') as fd:
    for i, fname in enumerate(data.iloc[:,0][train_index:]):
        fd.write('{}|{}\n'.format(os.path.join(os.getcwd(), fname), data.iloc[:,1][i]))

In [ ]:
%cat testing.txt

/content/dataset/wav/tts_lines_part_16_line_01160.wav|Con khốn nạn
/content/dataset/wav/tts_lines_part_14_line_01078.wav|Đồ đĩ đạc
/content/dataset/wav/tts_lines_part_13_line_01765.wav|Tao làm gì
/content/dataset/wav/tts_lines_part_07_line_00663.wav|Mịch tao đã làm gì mày để mày phụ tao
/content/dataset/wav/tts_lines_part_07_line_01655.wav|Nói
/content/dataset/wav/tts_lines_part_15_line_00544.wav|Nói mau
/content/dataset/wav/tts_lines_part_00_line_01688.wav|Nói
/content/dataset/wav/tts_lines_part_07_line_00879.wav|Hai bàn tay Long bóp cổ Mịch mà lắc lắc như người ta sóc ống thẻ xin thẻ trước điện thờ thánh Mịch lả oằn người đi như không còn xương sống ngã ngồi xuống làm Long cũng ngã theo và do cái ấy mới buông tay ra
/content/dataset/wav/tts_lines_part_18_line_01712.wav|Nước mắt ràn rụa Mịch nức nở nói
/content/dataset/wav/tts_lines_part_06_line_01096.wav| Anh vẫn yêu tôi như thế này ư
/content/dataset/wav/tts_lines_part_11_line_00658.wav|Anh vẫn giận tôi đến thế này ư
/content/datase

In [ ]:
_letters  = '0123456789aáảàãạâấẩầẫậăắẳằẵặbcdđeéẻèẽẹêếểềễệfghiíỉìĩịjklmnoóỏòõọôốổồỗộơớởờỡợpqrstuúủùũụưứửừữựvwxyýỷỳỹỵz'

In [ ]:
!pip install numpy==1.16.4

     |████████████████████████████████| 17.3MB 201kB/s 
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.16.4 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.0.0 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: seaborn 0.11.0 has requirement matplotlib>=2.2, but you'll have matplotlib 2.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.0.0 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.1.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: cvxpy 1.0.31 has requirement scipy>=1.1.0, but you'll have scipy

 - WARNING: The following packages were previously imported in this runtime:
  [numpy]
You must restart the runtime in order to use newly installed versions.

In [ ]:
# !cp drive/My\ Drive/Colab\ Notebooks/Speech-to-text/output_vi.zip /content/
!cp drive/My\ Drive/Colab\ Notebooks/Speech-to-text/output_vi_last.zip /content/

In [ ]:
# !unzip output_vi.zip
!unzip output_vi_last.zip

Archive:  output_vi_last.zip
  inflating: output_vi/checkpoint_7000  


In [ ]:
!rm output_vi.zip

In [ ]:
import os
import sys
checkpoint = sorted(os.listdir('output_vi/'))
checkpoint_path = 'output_vi/' + checkpoint[-1]

command = "python tacotron2/train.py -c " + '\'' + checkpoint_path + '\'' +  " -o output_vi -l logs"

os.system(command)

2

In [ ]:
!python tacotron2/train.py -c 'output_vi/checkpoint_7000' -o output_vi -l logs

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Train loss 7063 0.251323 Grad Norm 0.618217 5.11s/it
Train loss 7064 0.184940 Grad Norm 0.323851 4.08s/it
Train loss 7065 0.196629 Grad Norm 0.440631 5.31s/it
Train loss 7066 0.270828 Grad Norm 0.754318 3.45s/it
Train loss 7067 0.215672 Grad Norm 0.579044 4.66s/it
Train loss 7068 0.223711 Grad Norm 0.606657 2.83s/it
Train loss 7069 0.152189 Grad Norm 0.554501 4.76s/it
Train loss 7070 0.151165 Grad Norm 0.420428 5.91s/it
Train loss 7071 0.247254 Grad Norm 0.576356 4.67s/it
Train loss 7072 0.207806 Grad Norm 0.573871 4.12s/it
Train loss 7073 0.232932 Grad Norm 0.543503 3.79s/it
Train loss 7074 0.243821 Grad Norm 0.685152 3.18s/it
Train loss 7075 0.252259 Grad Norm 0.576579 2.66s/it
Train loss 7076 0.243870 Grad Norm 0.752920 3.04s/it
Train loss 7077 0.228127 Grad Norm 1.008655 4.46s/it
Train loss 7078 0.203452 Grad Norm 0.754166 3.75s/it
Train loss 7079 0.279088 Grad Norm 0.637826 2.74s/it
Train loss 7080 0.236985 Grad Norm 1.18134

In [ ]:
!zip -r output_vi_last.zip output_vi/checkpoint_12000

  adding: output_vi/checkpoint_12000 (deflated 6%)


In [ ]:
!ls -l --block-size=M output_vi/

total 1939M
-rw-r--r-- 1 root root 324M Oct 26 15:03 checkpoint_10000
-rw-r--r-- 1 root root 324M Oct 26 16:14 checkpoint_11000
-rw-r--r-- 1 root root 324M Oct 26 17:25 checkpoint_12000
-rw-r--r-- 1 root root 324M Oct 25 05:11 checkpoint_7000
-rw-r--r-- 1 root root 324M Oct 26 12:41 checkpoint_8000
-rw-r--r-- 1 root root 324M Oct 26 13:52 checkpoint_9000
drwxr-xr-x 2 root root   1M Oct 26 11:30 logs


In [ ]:
!cp output_vi_last.zip drive/My\ Drive/Colab\ Notebooks/Speech-to-text/
!echo 'ok go!'

ok go!


In [ ]:
!cp output_vi.zip drive/My\ Drive/Colab\ Notebooks/Speech-to-text/
!echo 'ok go!'

ok go!
